<a href="https://colab.research.google.com/github/Harin99/Event_Dataset_Preprocess/blob/main/MVSEC_Outdoor_day2__Preprocess.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import numpy as np
import os
import sys
import h5py
import pickle
import torch 

In [ ]:
data = h5py.File('/share/data/outdoor_day2/outdoor_day2_data.hdf5', 'r')
data_gt = h5py.File('/share/data/outdoor_day2/outdoor_day2_gt.hdf5', 'r')
processed_data = h5py.File('/share/data/outdoor_day2/proceseed_data.hdf5', 'r')

In [ ]:
print(data.keys())
print(data_gt.keys())
print(processed_data.keys())

## Flow Data & Timestamp Extraction 

In [2]:
def traverse_datasets(hdf_file):

    def h5py_dataset_iterator(g, prefix=''):
        for key in g.keys():
            item = g[key]
            path = f'{prefix}/{key}'
            if isinstance(item, h5py.Dataset): # test for dataset
                yield (path, item)
            elif isinstance(item, h5py.Group): # test for group (go down)
                yield from h5py_dataset_iterator(item, path)

    for path, what in h5py_dataset_iterator(hdf_file):
        yield path, what

In [3]:
with h5py.File('/share/data/outdoor_day2/outdoor_day2_gt.hdf5', 'r') as f:
    # get a h5py dataset object
    flow_dist = f['davis']['left']['flow_dist']
    print('flow_dist dtype :', flow_dist.dtype, '\n flow_dist shape :', flow_dist.shape)
    flow_dist_ts = f['davis']['left']['flow_dist_ts']
    print('flow_dist_ts dtype : ', flow_dist_ts.dtype, '\n flow_dist_ts shape :', flow_dist_ts.shape)
    
    # get the all data 
    flow_dist_data = flow_dist[:]
    flow_dist_ts_data = flow_dist_ts[:]

## Event Data Extraction 

In [ ]:
with h5py.File('/share/data/outdoor_day2//outdoor_day2_data.hdf5', 'r') as f:
    # get a h5py dataset object
    events = f['davis']['left']['events']
    print('events dtype :', events.dtype, '\n events shape :', events.shape)
    
    # get the all data 
    # ( x, y, ts, p )
    event_data = events[:]

## Optical Flow Timestamp에 대응되는 Event .npy Data 생성 

Optical Flow : 20 Hz (Original) \
optical flow timestamp : (12197,) \
Event timestamp : (466410069, 4) \
\
생성 방법 : [(t-1) optical flow ts] ~ [(t) optical flow ts] = (t) Event .npy \
첫번째 데이터 : 이전 모든 Event data ~ flow timestamp 까지의 모든 데이터를 넣기때문에 사용할 수 없는 더미 데이터 

In [4]:
events_list = []
saved_events_path = '/share/data/mvsec_outdoor_day_2_20Hz/outdoor_day_2/davis/left/events/'
saved_flow_path = '/share/data/mvsec_outdoor_day_2_20Hz/outdoor_day_2/optical_flow/'

for i, ts in enumerate(flow_dist_ts_data):
    if i == 0:
        # event_data[:, 2] : event timestamp 
        inner = ((ts - event_data[:, 2]) < 0.005 ) & (0 < (ts - event_data[:, 2]))
        inner = np.array(inner, dtype = float)
        inner_idx = np.nonzero(inner)[0][-1]
        events = event_data[:inner_idx+1, :]

        # Print
        print(events.shape)
        print("%s : %f ~ %f" %(str(i), events[0, 2], events[-1, 2]))

        prev_ts = inner_idx
        # Data Save
        np.save(saved_events_path + str(int(i)), events)
    
    else:
        # event_data[:, 2] : event timestamp 
        inner = ((ts - event_data[:, 2]) < 0.005 ) & (0 < (ts - event_data[:, 2]))
        inner = np.array(inner, dtype = float)
        inner_idx = np.nonzero(inner)[0][-1]
        events = event_data[prev_ts+1:inner_idx+1, :]

        # Print
        print(events.shape)
        print("%s : %f ~ %f" %(str(i), events[0, 2], events[-1, 2]))

        prev_ts = inner_idx
        # Data Save
        np.save(saved_events_path + str(int(i)), events)
    

## Optical Flow .npy Data 생성 

In [ ]:
for i, flow in enumerate(flow_dist_data):
    # Data 저장 
    print(flow.shape)
    np.save(saved_flow_path + str(int(i)), flow)

## flowtimestamps.txt 생성 
E-Raft Model에서는 해당 text 파일을 이용하여 data align을 진행함 

In [ ]:
saved_ts_path = '/share/data/mvsec_outdoor_day_2_20Hz/outdoor_day_2/'
np.savetxt(saved_ts_path + 'timestamps_flow.txt', flow_dist_ts_data)

## 최종 생성된 Data 확인 

In [ ]:
event_path = '/share/data/mvsec_outdoor_day_2_20Hz/outdoor_day_2/davis/left/events'
flow_path = '/share/data/mvsec_outdoor_day_2_20Hz/outdoor_day_2/optical_flow'
flow_ts_path = '/share/dataa/mvsec_outdoor_day_2_20Hz/outdoor_day_2/timestamps_flow.txt'

In [ ]:
event_list = os.listdir(event_path)
event_list[-1]

In [ ]:
flow_list = os.listdir(flow_path)
len(flow_list)
flow_list[-1]

In [ ]:
flow_ts = np.loadtxt(flow_ts_path)
len(flow_ts)
flow_ts[-1]